### FastText词向量
注：该任务语料库较小，用fastText效果比传统word2vec要好    

#### 功能函数预定义

In [ ]:
import jieba
import re


def tokenize(text):
    """
    带有语料清洗功能的分词函数, 包含数据预处理, 可以根据自己的需求重载
    """
    text = re.sub("\{%.+?%\}", " ", text)           # 去除 {%xxx%} (地理定位, 微博话题等)
    text = re.sub("@.+?( |$)", " ", text)           # 去除 @xxx (用户名)
    text = re.sub("【.+?】", " ", text)              # 去除 【xx】 (里面的内容通常都不是用户自己写的)
    icons = re.findall("\[.+?\]", text)             # 提取出所有表情图标
    text = re.sub("\[.+?\]", "IconMark", text)      # 将文本中的图标替换为`IconMark`

    tokens = []
    for k, w in enumerate(jieba.lcut(text)):
        w = w.strip()
        if "IconMark" in w:                         # 将IconMark替换为原图标
            for i in range(w.count("IconMark")):
                tokens.append(icons.pop(0))
        elif w and w != '\u200b' and w.isalpha():   # 只保留有效文本
                tokens.append(w)
    return tokens


def load_curpus(path):
    """
    加载语料库
    """
    data = []
    with open(path, "r", encoding="utf8") as f:
        for line in f:
            [_, seniment, content] = line.split(",", 2)
            content = tokenize(content)             # 分词
            data.append((content, int(seniment)))
    return data

#### 对语料库进行清洗和分词

In [2]:
import pandas as pd

data = load_curpus("weibo2018/train.txt") + load_curpus("weibo2018/test.txt")
df = pd.DataFrame(data, columns=["content", "sentiment"])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\GHOSTA~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.569 seconds.
Prefix dict has been built successfully.


In [3]:
df.head()

,content,sentiment
0,"[书中, 自有, 黄金屋, 书中, 自有, 颜如玉, 沿着, 岁月, 的, 长河, 跋涉, ...",1
1,"[这是, 英超, 被, 黑, 的, 最惨, 的, 一次, [二哈], [二哈], 十几年来,...",0
2,"[中国, 远洋, 海运, 集团, 副总经理, 俞曾, 港, 月, 日, 在, 上, 表示, ...",1
3,"[看, 流星花园, 其实, 也, 还好, 啦, 现在, 的, 观念, 以及, 时尚, 眼光,...",1
4,"[汉武帝, 的, 罪己, 诏, 的, 真实性, 尽管, 存在, 着, 争议, 然而, 轮台,...",1


#### 训练词向量

In [8]:
from gensim.models import FastText
model = FastText(df["content"], 
                 vector_size=100,
                 window=5, 
                 min_count=3, # 只保留出现次数大于3的词语
                 epochs=1000,  # 1000次训练
                 min_n=2,     # 默认为3,因为文本是中文这里改为2
                 max_n=4,     # 默认为6,因为文本是中文这里改为5
                 word_ngrams=1)

#### 保存模型

In [ ]:
model.save("model/model_100.txt")